In [ ]:
from FiinQuantX import FiinSession


username = 'DSTC_20@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(
    username=username,
    password=password,
).login()

In [ ]:
import pandas as pd
from FiinQuantX import FiinSession

# --- 1. Login ---
client = FiinSession(username=username, password=password).login()

# --- 2. Lấy toàn bộ tickers ---
hose_tickers = list(client.TickerList(ticker="VNINDEX"))
hnx_tickers  = list(client.TickerList(ticker="HNXINDEX"))
upcom_tickers = list(client.TickerList(ticker="UPCOMINDEX"))

all_tickers = hose_tickers + hnx_tickers + upcom_tickers
print(f"✅ Tổng số tickers: {len(all_tickers)}")

# Lưu danh sách tickers ra file
pd.DataFrame(all_tickers, columns=["ticker"]).to_csv("all_tickers.csv", index=False)
all_tickers = pd.read_csv("all_tickers.csv")["ticker"].tolist()
print("✅ Xuất file all_tickers.csv thành công")

# --- 3. Hàm flatten ratios ---
def flatten_ratios(fi_dict):
    rows = []
    for rec in fi_dict:
        base = {
            "ticker": rec.get("ticker"),
            "year": rec.get("year"),
            "quarter": rec.get("quarter")
        }
        ratios = rec.get("ratios", {})
        for group, metrics in ratios.items():
            for k, v in metrics.items():
                base[k] = v
        rows.append(base)
    return rows

# --- 4. Lấy dữ liệu cho tất cả tickers ---
all_rows = []
for i, t in enumerate(all_tickers, 1):
    try:
        fi_dict = client.FundamentalAnalysis().get_ratios(
            tickers=[t],
            TimeFilter="Yearly",
            LatestYear=2025,
            NumberOfPeriod=3,
            Consolidated=True
        )
        rows = flatten_ratios(fi_dict)
        all_rows.extend(rows)
        if i % 50 == 0:
            print(f"Đã xử lý {i}/{len(all_tickers)} tickers...")
    except Exception as e:
        print(f"⚠️ Lỗi khi lấy {t}: {e}")

# --- 5. Tạo DataFrame & Xuất CSV ---
df = pd.DataFrame(all_rows)
df = df.sort_values(by=["ticker", "year"]).reset_index(drop=True)

df.to_csv("all_fundamental_ratios.csv", index=False, encoding="utf-8-sig")
print("✅ Xuất file all_fundamental_ratios.csv thành công")


In [ ]:
def try_ratios(ticker):
    for consolidated in [True, False]:
        for year in [2025, 2024, 2023]:
            try:
                fi_dict = client.FundamentalAnalysis().get_ratios(
                    tickers=[ticker],
                    TimeFilter="Yearly",
                    LatestYear=year,
                    NumberOfPeriod=3,
                    Consolidated=consolidated
                )
                if fi_dict:
                    print(f"✅ Có dữ liệu {ticker}, year={year}, consolidated={consolidated}")
                    return fi_dict
            except:
                pass
    print(f"⚠️ Không có dữ liệu cho {ticker}")
    return None

test_data = try_ratios("ACB")


✅ Có dữ liệu ACB, year=2025, consolidated=True


In [ ]:
import pandas as pd

# Đọc lại file CSV đã xuất
df = pd.read_csv("all_fundamental_ratios.csv")

# Xác định cột dữ liệu ratios (bỏ ticker, year, quarter)
ratio_cols = [c for c in df.columns if c not in ["ticker", "year", "quarter"]]

# Tìm ticker nào tất cả giá trị ratios đều null
null_tickers = (
    df.groupby("ticker")[ratio_cols]
      .apply(lambda x: x.isnull().all().all())
)

# Lọc ra những ticker bị null hoàn toàn
null_tickers = null_tickers[null_tickers].index.tolist()

print("⚠️ Các ticker không có dữ liệu ratios:")
print(null_tickers)
print(f"👉 Tổng cộng: {len(null_tickers)} mã")


⚠️ Các ticker không có dữ liệu ratios:
['ABB', 'ACB', 'BAB', 'BID', 'BVB', 'CTG', 'EIB', 'EVF', 'HDB', 'HTS', 'KLB', 'LPB', 'MBB', 'MDD', 'MSB', 'NAB', 'NVB', 'OCB', 'PGB', 'PXK', 'SGB', 'SHB', 'SSB', 'STB', 'TCB', 'TIN', 'TPB', 'VAB', 'VBB', 'VCB', 'VIB', 'VPB']
👉 Tổng cộng: 32 mã


In [ ]:
import pandas as pd

# Đọc dữ liệu gốc
df = pd.read_csv("all_fundamental_ratios.csv")

# Xác định cột dữ liệu ratios (bỏ ticker, year, quarter)
ratio_cols = [c for c in df.columns if c not in ["ticker", "year", "quarter"]]

# Nhóm theo ticker và check có toàn null không
null_tickers = (
    df.groupby("ticker")[ratio_cols]
      .apply(lambda x: x.isnull().all().all())
)

# Tách ticker null và non-null
tickers_null = null_tickers[null_tickers].index.tolist()
tickers_non_null = null_tickers[~null_tickers].index.tolist()

# Tạo 2 DataFrame
df_null = df[df["ticker"].isin(tickers_null)]
df_non_null = df[df["ticker"].isin(tickers_non_null)]

# Xuất ra CSV
df_null.to_csv("ratios_null.csv", index=False, encoding="utf-8-sig")
df_non_null.to_csv("ratios_non_null.csv", index=False, encoding="utf-8-sig")

print(f"✅ Xuất xong!")
print(f"👉 ratios_null.csv: {len(df_null['ticker'].unique())} mã")
print(f"👉 ratios_non_null.csv: {len(df_non_null['ticker'].unique())} mã")


✅ Xuất xong!
👉 ratios_null.csv: 32 mã
👉 ratios_non_null.csv: 1579 mã


In [ ]:
import pandas as pd
from vietfin import vf

bank_tickers = [
    'ABB', 'ACB', 'BAB', 'BID', 'BVB', 'CTG', 'EIB', 'EVF', 'HDB', 'HTS',
    'KLB', 'LPB', 'MBB', 'MDD', 'MSB', 'NAB', 'NVB', 'OCB', 'PGB', 'PXK',
    'SGB', 'SHB', 'SSB', 'STB', 'TCB', 'TIN', 'TPB', 'VAB', 'VBB', 'VCB',
    'VIB', 'VPB'
]

all_df = []

for ticker in bank_tickers:
    try:
        ratios_obj = vf.equity.fundamental.ratios(symbol=ticker)

        # Truy xuất DataFrame từ VfObject
        if hasattr(ratios_obj, "to_df"):
            df = ratios_obj.to_df()
        elif hasattr(ratios_obj, "data"):
            df = pd.DataFrame(ratios_obj.data)
        else:
            raise ValueError("Không tìm thấy dữ liệu dạng bảng")

        if not df.empty:
            df["ticker"] = ticker
            all_df.append(df)
            print(f"✅ Lấy ratios thành công: {ticker}, {len(df)} dòng")
        else:
            print(f"⚠️ Không có dữ liệu cho {ticker}")

    except Exception as e:
        print(f"❌ Lỗi với {ticker}: {e}")

if all_df:
    df_all = pd.concat(all_df, ignore_index=True)
    df_all.to_csv("bank_ratios_vietfin.csv", index=False, encoding="utf-8-sig")
    print(f"📂 Đã lưu file bank_ratios_vietfin.csv ({df_all.shape[0]} dòng)")
else:
    print("❌ Không có dữ liệu nào được lưu")


Retrieved 17 financial ratios data point for stock ticker ABB.
✅ Lấy ratios thành công: ABB, 17 dòng
Retrieved 21 financial ratios data point for stock ticker ACB.
✅ Lấy ratios thành công: ACB, 21 dòng
Retrieved 14 financial ratios data point for stock ticker BAB.
✅ Lấy ratios thành công: BAB, 14 dòng
Retrieved 20 financial ratios data point for stock ticker BID.
✅ Lấy ratios thành công: BID, 20 dòng
Retrieved 6 financial ratios data point for stock ticker BVB.
✅ Lấy ratios thành công: BVB, 6 dòng
Retrieved 18 financial ratios data point for stock ticker CTG.
✅ Lấy ratios thành công: CTG, 18 dòng
Retrieved 19 financial ratios data point for stock ticker EIB.
✅ Lấy ratios thành công: EIB, 19 dòng
Retrieved 5 financial ratios data point for stock ticker EVF.
✅ Lấy ratios thành công: EVF, 5 dòng
Retrieved 18 financial ratios data point for stock ticker HDB.
✅ Lấy ratios thành công: HDB, 18 dòng
❌ Lỗi với HTS: No data found for the symbol HTS.
Retrieved 18 financial ratios data point for s